In [1]:
using MLDataPattern
using Plots; pyplot()
using ProgressMeter
using JLD
import PyCall
using DrakeVisualizer, GeometryTypes, CoordinateTransformations
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

true

In [2]:
include("../../pympc.jl")
colmat = PyMPC.colmat

Test Summary:    | Pass  Total
test sensitivity |  267    267


In [3]:
sys = PyMPC.Models.BoxAtlas()

PyMPC.Models.BoxAtlas(PyObject <pympc.models.boxatlas.BoxAtlasPWAModel object at 0x7f6aedf1d310>)

In [4]:
vis = Visualizer()[:boxatlas]
setgeometry!(vis, sys)

Set{Array{Symbol,1}}()

In [5]:
controller = PyMPC.Models.controller(sys, Q=10*eye(10))

PyObject <pympc.control.MPCHybridController instance at 0x7f6b1469e6c8>

In [6]:
struct Sample
    utraj::Vector{Vector{Float64}}
    xtraj::Vector{Vector{Float64}}
    switching_sequence::Vector{Int}
    u_sensitivity::Matrix{Float64}
end

In [ ]:
num_samples = 100
num_sim_steps = 20
data = Vector{Sample}()
# data = Vector{Tuple{Vector{Float64}, Matrix{Float64}}}()

@showprogress for i in 1:num_samples
    while true
        x0 = PyMPC.Models.generate_x0(sys)
        settransform!(vis, sys, x0)
        utraj, xtraj, switching_sequence, J = PyMPC.run_mpc(controller, x0)
        u = utraj[1]
        if !isnan(u[1])
            @assert x0 == xtraj[1]
            PyMPC.Models.playback(vis, sys, xtraj, 1)
            push!(data, Sample(utraj, xtraj, collect(switching_sequence), J[1:length(u), :]))
            for j in 1:num_sim_steps
                x = xtraj[2]
                utraj = vcat(utraj[2:end], [zeros(utraj[end])])
                xnext = try
                    vec(sys.sys[:pwa_system][:simulate](colmat(xtraj[end]), [colmat(utraj[end-1])])[1][2])
                catch e
                    if isa(e, PyCall.PyError)
                        zeros(xtraj[end])
                    else
                        rethrow(e)
                    end
                end
                xtraj = vcat(xtraj[2:end], [xnext])
                switching_sequence = (switching_sequence[2:end]..., switching_sequence[end])
                utraj, xtraj, switching_sequence, J = PyMPC.run_mpc(controller, x, utraj, xtraj, switching_sequence)
                u = utraj[1]
                if isnan(u[1])
                    break
                end
                @assert x == xtraj[1]
                push!(data, Sample(utraj, xtraj, collect(switching_sequence), J[1:length(u), :]))
                PyMPC.Models.playback(vis, sys, xtraj, 1)
            end
            break
        end
    end
    save("box_atlas_100_traj.jld", "data", data)
end

save("box_atlas_100_traj.jld", "data", data)

Progress:   2%|█                                        |  ETA: 5:32:35